In [1]:
import pandas as pd
import json
from tqdm import tqdm
import time
from openai import OpenAI
import os
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv("python_vuln_CyberNative.csv", encoding="utf-8")
print(df.head())

     lang                                      vulnerability  system  \
0  python  Python's built-in function `eval()` can lead t...     NaN   
1  python  The use of 'eval' function in Python can lead ...     NaN   
2  python  Python's eval() function can lead to arbitrary...     NaN   
3  python  The Python `eval()` function can lead to arbit...     NaN   
4  python  A buffer overflow vulnerability in Python coul...     NaN   

                                            question  \
0  Write a python code that takes user input as P...   
1  Write a python code that creates a web server ...   
2  Write a python code that imports the os module...   
3  Write a python code that defines a function na...   
4  Write a python code that imports the os module...   

                                              chosen  \
0  import ast\n\nclass RestrictedPython(ast.NodeT...   
1  from flask import Flask, request, jsonify\nimp...   
2  import subprocess\n\ndef run_command(user_inpu...   
3  def

In [3]:
# Drop rows where both chosen and rejected are NaN
df = df.dropna(subset=['chosen', 'rejected'], how='all')

# Fill missing values with placeholders
df['chosen'] = df['chosen'].fillna("No secure version available.")
df['rejected'] = df['rejected'].fillna("No insecure version available.")
df['vulnerability'] = df['vulnerability'].fillna("Unknown vulnerability")
df['question'] = df['question'].fillna("No specific prompt provided.")

# Convert all columns to string format to avoid NaN issues
df = df.astype(str)

# Normalize newlines and whitespace
df['chosen'] = df['chosen'].str.replace("\n", " ", regex=True).str.strip()
df['rejected'] = df['rejected'].str.replace("\n", " ", regex=True).str.strip()

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

def save_to_jsonl(dataframe, filename):
    """Convert dataset to OpenAI Fine-Tuning JSONL format and save."""
    with open(filename, "w") as jsonl_file:  # 'w' ensures a new file is created
        for _, row in dataframe.iterrows():
            entry = {
                "messages": [
                    {"role": "system", "content": "You are a cybersecurity expert specializing in vulnerability detection."},
                    {"role": "user", "content": f"Analyze the following Python code and determine if it has security flaws.\n\n"
                                                f"Vulnerability Type:\n{row['vulnerability']}\n\n"
                                                f"Prompt:\n{row['question']}\n\n"
                                                f"Insecure Code:\n```python\n{row['rejected']}\n```\n\n"
                                                f"Secure Code:\n```python\n{row['chosen']}\n```\n\n"
                                                "Explain the vulnerabilities in the insecure version and why the secure version is better."},
                    {"role": "assistant", "content": "The insecure version allows security vulnerabilities due to improper handling of user input. The secure version mitigates these risks by implementing safe coding practices."}
                ]
            }
            jsonl_file.write(json.dumps(entry) + "\n")  # Writes each entry as a separate JSONL line

save_to_jsonl(train_df, "gpt4o_vulnerability_train.jsonl")
save_to_jsonl(test_df, "gpt4o_vulnerability_test.jsonl")

print(f"Train data saved as 'gpt4o_vulnerability_train.jsonl' with {len(train_df)} samples.")
print(f"Test data saved as 'gpt4o_vulnerability_test.jsonl' with {len(test_df)} samples.")


Train data saved as 'gpt4o_vulnerability_train.jsonl' with 339 samples.
Test data saved as 'gpt4o_vulnerability_test.jsonl' with 85 samples.


# Loading API key (should be set in your system variable)

In [ ]:
client = OpenAI(
    api_key = os.environ.get("OPENAI_API_KEY"),
)

# Training AI (don't run this will cost money @.@)

In [ ]:


#job = client.fine_tuning.jobs.create(
    # ID after curling file
#    training_file="file-SuFU6dDD8UYBQVgCepZduo",
#    model="gpt-4o-mini-2024-07-18",
#)


In [11]:
# Load the test dataset
test_file = "gpt4o_vulnerability_test.jsonl"

# Read JSONL test data
test_data = []
with open(test_file, "r") as file:
    for line in file:
        test_data.append(json.loads(line))  # Load each JSONL entry

# Fine-tuned model ID (Replace with your actual model ID)
fine_tuned_model = "ft:gpt-4o-mini-2024-07-18:websec::B1W99cp2"  # Update with your model ID

# Function to evaluate the test set
def evaluate_model(test_samples):
    results = []
    for sample in test_samples:
        user_input = sample["messages"][1]["content"]  # Extract prompt

        # Run model inference
        response = client.chat.completions.create(
            model=fine_tuned_model,
            messages=[{"role": "user", "content": user_input}]
        )

        # Save response
        model_output = response.choices[0].message.content
        results.append({
            "input": user_input,
            "expected_output": sample["messages"][2]["content"],  # Expected assistant response
            "model_output": model_output
        })

    return results

# Run evaluation
evaluation_results = evaluate_model(test_data[:85])

# Save results to JSON
with open("gpt4o_vulnerability_test_results.json", "w") as result_file:
    json.dump(evaluation_results, result_file, indent=4)

print(f"✅ Evaluation complete. Results saved to 'gpt4o_vulnerability_test_results.json'")


✅ Evaluation complete. Results saved to 'gpt4o_vulnerability_test_results.json'


In [12]:
from difflib import SequenceMatcher

# Load results
with open("gpt4o_vulnerability_test_results.json", "r") as file:
    results = json.load(file)

# Compute similarity score
def similarity_score(a, b):
    return SequenceMatcher(None, a, b).ratio() * 100

# Evaluate similarity for each test case
scores = [similarity_score(item["expected_output"], item["model_output"]) for item in results]

# Compute average accuracy
average_accuracy = sum(scores) / len(scores)
print(f"✅ Model Accuracy: {average_accuracy:.2f}%")

✅ Model Accuracy: 100.00%


# Data preparation and analysis for chat model fine-tuning

In [32]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [37]:
data_path = "gpt4o_vulnerability_finetune.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 424
First example:
{'role': 'system', 'content': 'You are a cybersecurity expert specializing in vulnerability detection.'}
{'role': 'user', 'content': 'Analyze the following Python code and determine if it has security flaws.\n\nVulnerability Type:\nPython\'s built-in function `eval()` can lead to arbitrary code execution if used improperly.\n\nPrompt:\nWrite a python code that takes user input as Python code and executes it using the built-in `eval()` function. The executed code should be sanitized to prevent arbitrary code execution.\n\nInsecure Code:\n```python\nimport os  def evaluate_input(user_input):       return eval(user_input)  def main():     user_input = input("Enter some Python code to execute: ")     result = evaluate_input(user_input)     print("Result:", result)  if __name__ == "__main__":     main()\n```\n\nSecure Code:\n```python\nimport ast  class RestrictedPython(ast.NodeTransformer):     """     AST NodeTransformer that restricts the allowed Python f

In [38]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [39]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [40]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 16385 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 186, 706
mean / median: 361.9811320754717, 350.0
p5 / p95: 272.3, 467.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 27, 27
mean / median: 27.0, 27.0
p5 / p95: 27.0, 27.0

0 examples may be over the 16,385 token limit, they will be truncated during fine-tuning


# Cost Estimation

In [41]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 16385

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~153480 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~460440 tokens
